<a href="https://colab.research.google.com/github/Jderrick97/Colab/blob/main/Assingment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As you can see I did not fully run this version of my code in Colab. However, I ran it locally on my own PC and it works. I just did not have time to run it again in Colab.

In [ ]:
############ Imports ############
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms import Grayscale, Resize, ToTensor
from PIL import Image
import os

In [ ]:
############ Link Google Drive ############
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
############ Unzip the file ############
!pip install py7zr
import py7zr

import py7zr
with py7zr.SevenZipFile("/content/drive/MyDrive/Colab Notebooks/data.7z", mode='r') as z:
    z.extractall()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
############ Convolutional Autoencoder ############
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.LeakyReLU(0.1),            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2, padding=0),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(64, 128, 3, padding=1),  
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2, padding=0),
            nn.Conv2d(128, 128, 3, padding=1), 
            nn.LeakyReLU(0.1),
            nn.Conv2d(128, 256, 3, padding=1), 
            nn.LeakyReLU(0.1),)
        
        
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1),  
            nn.LeakyReLU(0.1),
            nn.Upsample(scale_factor=2, mode='bicubic'),
            nn.Conv2d(128, 64, 3, padding=1),  
            nn.LeakyReLU(0.1),
            nn.Conv2d(64, 32, 3, padding=1),  
            nn.LeakyReLU(0.1),
            nn.Upsample(scale_factor=2, mode='bicubic'),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.LeakyReLU(0.1),
            nn.Upsample(scale_factor=2, mode='bicubic'),
            nn.Conv2d(32, 1, 3, padding=1),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = Autoencoder()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
############ Setting up the Dataset ############
class DogsCatsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_names = os.listdir(root_dir)

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_path = os.path.join(self.root_dir, self.file_names[idx])
        image = Image.open(img_path).convert('L')
        if self.transform:
            image = self.transform(image)

        return image, 0

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()])


train_path = "train"
test_path = "test1"
train_dataset = DogsCatsDataset(train_path, transform=transform)
test_dataset = DogsCatsDataset(test_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
############ Training the Model ############
autoencoder = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
resizer = transforms.Resize((56, 56))

epochs = 50


for epoch in range(epochs):
    for pic, _ in train_loader:
      pic = pic.to(device)
      noisy_inputs = pic + 0.5 * torch.randn(pic.size()).to(device)  
      noisy_inputs = torch.clamp(noisy_inputs, 0., 1.) 
      optimizer.zero_grad()
      outputs = autoencoder(noisy_inputs)
      target = resizer(pic)
      loss = criterion(outputs, target)
      loss.backward()
      optimizer.step()
    print(f'Epoch [{epoch+1}/{50}], loss: {loss.item():.4f}')



# Save the model
torch.save(autoencoder.state_dict(), 'Autoencoder.pth')

/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
############ Comparing our Output ############
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_tensor, to_pil_image


model = Autoencoder()
model.load_state_dict(torch.load('Autoencoder.pth'))
model.eval()


orig_image_1 = Image.open('test1/69.jpg').convert('L')
resized_image_1 = orig_image_1.copy().resize((28, 28))
input_tensor_1 = to_tensor(resized_image_1).unsqueeze(0)


with torch.no_grad():
    output_tensor_1 = model(input_tensor_1)
output_image_1 = to_pil_image(output_tensor_1.squeeze(0))

orig_image_2 = Image.open('test1/1200.jpg').convert('L')
resized_image_2 = orig_image_2.copy().resize((28, 28))
input_tensor_2 = to_tensor(resized_image_2).unsqueeze(0)


with torch.no_grad():
    output_tensor_2 = model(input_tensor_2)
output_image_2 = to_pil_image(output_tensor_2.squeeze(0))

orig_image_3 = Image.open('test1/10000.jpg').convert('L')
resized_image_3 = orig_image_3.copy().resize((28, 28))
input_tensor_3 = to_tensor(resized_image_3).unsqueeze(0)


with torch.no_grad():
    output_tensor_3 = model(input_tensor_3)
output_image_3 = to_pil_image(output_tensor_3.squeeze(0))




f, (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9) = plt.subplots(9, sharex=True, sharey=True)
ax1.imshow(resized_image_1, cmap = 'gray')
ax2.imshow(output_image_1, cmap = 'gray')
ax3.imshow(orig_image_2.copy().resize((56, 56)), cmap = 'gray')
ax4.imshow(resized_image_2, cmap = 'gray')
ax5.imshow(output_image_2, cmap = 'gray')
ax6.imshow(orig_image_2.copy().resize((56, 56)), cmap = 'gray')
ax7.imshow(resized_image_3, cmap = 'gray')
ax8.imshow(output_image_3, cmap = 'gray')
ax9.imshow(orig_image_3.copy().resize((56, 56)), cmap = 'gray')
plt.show()


FileNotFoundError: ignored

In [ ]:
############ Set the Colab run time to run continuously ############
import IPython
from google.colab import output

display(IPython.display.Javascript('''
  function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-toolbar-button#connect").click() 
  }
  setInterval(ClickConnect,60000)
'''))